<a href="https://colab.research.google.com/github/etgins/weapon_detection_with_transfer_learning/blob/main/DL_project_180122.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import shutil
from torch.utils.data import DataLoader, Dataset
import time
import copy
import cv2
from IPython.display import Image  # for displaying images
from sklearn.model_selection import train_test_split
from xml.dom import minidom

# %matplotlib notebook
%matplotlib inline

import xml.etree.ElementTree as ET

seed = 212
np.random.seed(seed)
torch.manual_seed(seed)

from google.colab import drive
drive._mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/046211_Deep_Learning/DL_project'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/046211_Deep_Learning/DL_project


# DATA PROCESSING

In [29]:
# Function to get the data from XML Annotation

def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [30]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"pistol": 0,
                           "knife": 1}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    # save_file_name = os.path.join("annotations2", info_dict["filename"].replace("JPG", "txt"))
    
    # Save the annotation to disk
    # print("\n".join(print_buffer), file= open(save_file_name, "w"))


In [31]:
# convert all annotations with a loop - run once to create annotations

# # Get the annotations
# annotations = [os.path.join('annotations', x) for x in os.listdir('./data/Knife_detection/annotations') if x[-3:] == "xml"]
# # annotations = [os.path.join(x) for x in os.listdir('./data/Pistol detection/xmls') if x[-3:] == "xml"]
# annotations.sort()
# print(annotations)

# print(annotations)
# # Convert and save the annotations

# for ann in tqdm(annotations):
#     print(ann)
#     info_dict = extract_info_from_xml(os.path.join('./data/Knife_detection', ann))
#     convert_to_yolov5(info_dict)
# annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]
# print(annotations)

In [32]:
# THIS IS TEMP AND ONE-TIME-ONLY
# ##########################


# def add_numbers(filename):
#     with open(filename, 'r') as readfile:
#         data = readfile.readlines()
#     with open(filename, 'w') as writefile:
#         for i, line in enumerate(data):
#             writefile.write('%s %s' % (line, '.txt'))

# filename = './annotations/armas (1)'
# add_numbers(filename)

# for path, _, filenames in os.walk(folder):
#     for filename in filenames:
#         add_numbers(os.path.join(path, filename))


### USE THIS TO ADD .txt TO ANNOTATIONS (IF NEEDED):
#############
# from pathlib import Path
# for file in range(3,3001): 
#   p = Path('./annotations/armas (' + str(file) + ')')
#   p.rename(p.with_suffix('.txt'))
#############


# THIS IS TEMP AND ONE-TIME-ONLY
# ##########################

In [33]:
!pwd

# for first time:
# ---------------
# annotations = os.listdir('annotations')
# annotations.sort()
# # print('annotations: ', annotations)
# images = os.listdir('images')
# images.sort()
# # print('images: ', images)
# ---------------

# for second time and on:
# ---------------
train_annotations = os.listdir('./labels/train')
train_annotations.sort()
train_images = os.listdir('./images/train')
train_images.sort()
# ---------------


class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))


# SANITY CHECK
def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)), outline ="red", width = 4)     # fill ="#ffff33"
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.imshow(np.array(image))
    plt.show()


# Get any random annotation file 
annotation_file = random.choice(train_annotations)
print('short annotation file:', annotation_file)
annotation_file = os.path.join('./labels/train', annotation_file)
# print('long annotation file:', annotation_file)

with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    # print('annotation file content:', annotation_list)
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

#Get the corresponding image file
image_file = annotation_file.replace("labels", "images").replace("txt", "jpg")
print('image file:', image_file)
# assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)
# image.show()
#Plot the Bounding Box
plot_bounding_box(image, annotation_list)


/content/drive/My Drive/Colab Notebooks/046211_Deep_Learning/DL_project
short annotation file: knife_1076.txt
image file: ./images/train/knife_1076.jpg


AttributeError: ignored

In [34]:
# Read images and annotations

# images = [x for x in os.listdir('images') if x[-3:] == "jpg"]
# annotations = [x for x in os.listdir('annotations') if x[-3:] == "txt"]


# option a: Split the dataset into train-valid-test splits - for first split (all in the same directory)
# --------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------
# images = [os.path.join('images', x) for x in os.listdir('images') if x[-3:] == 'jpg']
# annotations = [os.path.join('annotations', x) for x in os.listdir('annotations') if x[-3:] == "txt"]

# images.sort()
# annotations.sort()

# # print(images)
# # print(annotations)

# print(len(images))
# print(len(annotations))

# train_images, test_images, train_annotations, test_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
# val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

# --------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------





# option b: Split the dataset - if already split into separate directories:
# --------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------

train_images = [os.path.join('/images/train', x) for x in os.listdir('./images/train')  if x[-3:] == 'jpg' or x[-3:] == 'JPG']
test_images = [os.path.join('./images/test', x) for x in os.listdir('./images/test')  if x[-3:] == 'jpg' or x[-3:] == 'JPG']
train_annotations = [os.path.join('./labels/train', x) for x in os.listdir('./labels/train') if x[-3:] == "txt"]
test_annotations = [os.path.join('./labels/test', x) for x in os.listdir('./labels/test')if x[-3:] == "txt"]

train_images.sort()
test_annotations.sort()
train_annotations.sort()
test_images.sort()

print(len(train_images), 'train images: ', train_images)
print(len(train_annotations), 'train_annotations: ', train_annotations)
print(len(test_images),'test images: ', test_images)
print(len(test_annotations), 'test_annotations: ', test_annotations)


# --------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------

4061 train images:  ['/images/train/DSC_0001.JPG', '/images/train/DSC_00011.JPG', '/images/train/DSC_0002.JPG', '/images/train/DSC_00021.JPG', '/images/train/DSC_0003.JPG', '/images/train/DSC_00031.JPG', '/images/train/DSC_0004.JPG', '/images/train/DSC_00041.JPG', '/images/train/DSC_00051.JPG', '/images/train/DSC_0006.JPG', '/images/train/DSC_0007.JPG', '/images/train/DSC_00071.JPG', '/images/train/DSC_0008.JPG', '/images/train/DSC_00081.JPG', '/images/train/DSC_0009.JPG', '/images/train/DSC_00091.JPG', '/images/train/DSC_0010.JPG', '/images/train/DSC_00101.JPG', '/images/train/DSC_0011.JPG', '/images/train/DSC_00111.JPG', '/images/train/DSC_0012.JPG', '/images/train/DSC_00121.JPG', '/images/train/DSC_0013.JPG', '/images/train/DSC_00131.JPG', '/images/train/DSC_0014.JPG', '/images/train/DSC_00141.JPG', '/images/train/DSC_0015.JPG', '/images/train/DSC_00151.JPG', '/images/train/DSC_0016.JPG', '/images/train/DSC_00161.JPG', '/images/train/DSC_0017.JPG', '/images/train/DSC_00171.JPG', '/i

In [35]:
# !mkdir images/train images/test annotations/train annotations/test

In [36]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        print(f)
        try:
            shutil.move(f, destination_folder)
        except:
            print(f, 'failed')
            assert False


In [37]:
# # Move the splits into their folders

# move_files_to_folder(train_images, 'images/train')
# move_files_to_folder(train_annotations, 'annotations/train/')
# move_files_to_folder(test_images, 'images/test/')
# move_files_to_folder(test_annotations, 'annotations/test/')

# MODEL TRAINING

In [56]:
# RUN ONCE
%cd /content/drive/My Drive/Colab Notebooks/046211_Deep_Learning/DL_project/
# !git clone https://github.com/ultralytics/yolov5
# !git pull https://github.com/ultralytics/yolov5

/content/drive/My Drive/Colab Notebooks/046211_Deep_Learning/DL_project
Cloning into 'yolov5'...
remote: Enumerating objects: 10562, done.
remote: Total 10562 (delta 0), reused 0 (delta 0), pack-reused 10562
Receiving objects: 100% (10562/10562), 10.75 MiB | 8.00 MiB/s, done.
Resolving deltas: 100% (7297/7297), done.


In [50]:
!which pip

/usr/local/bin/pip


In [51]:
!pwd
%cd /content/drive/My Drive/Colab Notebooks/046211_Deep_Learning/DL_project/

/content/drive/My Drive/Colab Notebooks/046211_Deep_Learning/DL_project
/content/drive/My Drive/Colab Notebooks/046211_Deep_Learning/DL_project


In [52]:
!pip install -r yolov5/requirements.txt

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/046211_Deep_Learning/DL_project/yolov5
# !pip install wandb
!python train.py --img 640 --cfg yolov5m.yaml --hyp hyp.scratch.yaml --batch 32 --epochs 60 --data weapons.yaml --weights yolov5m.pt --workers 24 --name yolo_weapons_det --freeze 15

/content/drive/My Drive/Colab Notebooks/046211_Deep_Learning/DL_project/yolov5
wandb: Currently logged in as: etgins (use `wandb login --relogin` to force relogin)
train: weights=yolov5m.pt, cfg=yolov5m.yaml, data=weapons.yaml, hyp=hyp.scratch.yaml, epochs=60, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=24, project=runs/train, name=yolo_weapons_det, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[20], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 2022-1-18 torch 1.10.0+cu111 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.98, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_

# INFERENCE + DETECTION

In [55]:
!python detect.py --source ../images/test/ --weights runs/train/yolo_weapons_det/weights/best.pt --conf 0.25 --name yolo_weapons_det

python3: can't open file 'detect.py': [Errno 2] No such file or directory
